In [1]:
#Import the libraries 
import pandas as pd

# Load the dataset
influencers_data = pd.read_csv('top_insta_influencers_data.csv')

# Inspect the first few rows of the dataset
influencers_data.head()

,rank,channel_info,influence_score,posts,followers,avg_likes,60_day_eng_rate,new_post_avg_like,total_likes,country
0,1,cristiano,92,3.3k,475.8m,8.7m,1.39%,6.5m,29.0b,Spain
1,2,kyliejenner,91,6.9k,366.2m,8.3m,1.62%,5.9m,57.4b,United States
2,3,leomessi,90,0.89k,357.3m,6.8m,1.24%,4.4m,6.0b,NaN
3,4,selenagomez,93,1.8k,342.7m,6.2m,0.97%,3.3m,11.5b,United States
4,5,therock,91,6.8k,334.1m,1.9m,0.20%,665.3k,12.5b,United States


In [3]:
#Check the data:
influencers_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   rank               200 non-null    int64 
 1   channel_info       200 non-null    object
 2   influence_score    200 non-null    int64 
 3   posts              200 non-null    object
 4   followers          200 non-null    object
 5   avg_likes          200 non-null    object
 6   60_day_eng_rate    200 non-null    object
 7   new_post_avg_like  200 non-null    object
 8   total_likes        200 non-null    object
 9   country            138 non-null    object
dtypes: int64(2), object(8)
memory usage: 15.8+ KB


In [5]:
#Check the null values 
influencers_data.isna().sum()

rank                  0
channel_info          0
influence_score       0
posts                 0
followers             0
avg_likes             0
60_day_eng_rate       0
new_post_avg_like     0
total_likes           0
country              62
dtype: int64

In [7]:
# Drop rows where 'country' is NaN
influencers_data = influencers_data.dropna(subset=['country'])

In [9]:
# Function to convert values with k, m, b to numerical format
def convert_to_numeric(value):
    try:
        if isinstance(value, str):
            if 'k' in value:
                return float(value.replace('k', '')) * 1e3
            elif 'm' in value:
                return float(value.replace('m', '')) * 1e6
            elif 'b' in value:
                return float(value.replace('b', '')) * 1e9
            elif '%' in value:
                return float(value.replace('%', '')) / 100
        return float(value)
    except ValueError:
        return None

# Apply conversion to relevant columns
columns_to_convert = ['posts', 'followers', 'avg_likes', '60_day_eng_rate',
                      'new_post_avg_like', 'total_likes']

for column in columns_to_convert:
    influencers_data[column] = influencers_data[column].apply(convert_to_numeric)

In [11]:
#Check the data
influencers_data.head()

,rank,channel_info,influence_score,posts,followers,avg_likes,60_day_eng_rate,new_post_avg_like,total_likes,country
0,1,cristiano,92,3300.0,475800000.0,8700000.0,0.0139,6500000.0,2.900000e+10,Spain
1,2,kyliejenner,91,6900.0,366200000.0,8300000.0,0.0162,5900000.0,5.740000e+10,United States
3,4,selenagomez,93,1800.0,342700000.0,6200000.0,0.0097,3300000.0,1.150000e+10,United States
4,5,therock,91,6800.0,334100000.0,1900000.0,0.0020,665300.0,1.250000e+10,United States
5,6,kimkardashian,91,5600.0,329200000.0,3500000.0,0.0088,2900000.0,1.990000e+10,United States


In [13]:
df=influencers_data.copy()

In [15]:
# Coefficients selected to match the closely with the real estimated value for each influencer
alpha = 0.8
beta = 0.6
gamma = 0.8
delta = 0.0025

# Assumption: Influencers have been active for 52 weeks (1 year)
weeks_active = 52

# Deriving the columns
df['Followers_per_M'] = df['followers'] / 1000000
df['Posts_per_Week'] = df['posts'] / weeks_active  # Dividing by assumed active weeks
df['Engagement_Rate'] = df['60_day_eng_rate']  # Using the provided engagement rate over 60 days
df['Likes_per_Post'] = df['avg_likes']  # Using the average likes as likes per post

# Calculating Cost per Hour
df['Cost_per_Hour'] = (
    alpha * df['Followers_per_M'] +
    beta * df['Posts_per_Week'] +
    gamma * df['Engagement_Rate'] +
    delta * df['Likes_per_Post']
)

In [17]:
#Check the data
df

,rank,channel_info,influence_score,posts,followers,avg_likes,60_day_eng_rate,new_post_avg_like,total_likes,country,Followers_per_M,Posts_per_Week,Engagement_Rate,Likes_per_Post,Cost_per_Hour
0,1,cristiano,92,3300.0,475800000.0,8700000.0,0.0139,6500000.0,2.900000e+10,Spain,475.8,63.461538,0.0139,8700000.0,22168.728043
1,2,kyliejenner,91,6900.0,366200000.0,8300000.0,0.0162,5900000.0,5.740000e+10,United States,366.2,132.692308,0.0162,8300000.0,21122.588345
3,4,selenagomez,93,1800.0,342700000.0,6200000.0,0.0097,3300000.0,1.150000e+10,United States,342.7,34.615385,0.0097,6200000.0,15794.936991
4,5,therock,91,6800.0,334100000.0,1900000.0,0.0020,665300.0,1.250000e+10,United States,334.1,130.769231,0.0020,1900000.0,5095.743138
5,6,kimkardashian,91,5600.0,329200000.0,3500000.0,0.0088,2900000.0,1.990000e+10,United States,329.2,107.692308,0.0088,3500000.0,9077.982425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,iambeckyg,71,2300.0,33200000.0,623800.0,0.0140,464700.0,1.400000e+09,United States,33.2,44.230769,0.0140,623800.0,1612.609662
196,197,nancyajram,81,3800.0,33200000.0,390400.0,0.0064,208000.0,1.500000e+09,France,33.2,73.076923,0.0064,390400.0,1046.411274
197,198,luansantana,79,770.0,33200000.0,193300.0,0.0026,82600.0,1.492000e+08,Brazil,33.2,14.807692,0.0026,193300.0,518.696695
198,199,nickjonas,78,2300.0,33000000.0,719600.0,0.0142,467700.0,1.700000e+09,United States,33.0,44.230769,0.0142,719600.0,1851.949822


In [19]:
# Reset the index of the DataFrame
df = df.reset_index(drop=True)

In [21]:
#Check the data
df

,rank,channel_info,influence_score,posts,followers,avg_likes,60_day_eng_rate,new_post_avg_like,total_likes,country,Followers_per_M,Posts_per_Week,Engagement_Rate,Likes_per_Post,Cost_per_Hour
0,1,cristiano,92,3300.0,475800000.0,8700000.0,0.0139,6500000.0,2.900000e+10,Spain,475.8,63.461538,0.0139,8700000.0,22168.728043
1,2,kyliejenner,91,6900.0,366200000.0,8300000.0,0.0162,5900000.0,5.740000e+10,United States,366.2,132.692308,0.0162,8300000.0,21122.588345
2,4,selenagomez,93,1800.0,342700000.0,6200000.0,0.0097,3300000.0,1.150000e+10,United States,342.7,34.615385,0.0097,6200000.0,15794.936991
3,5,therock,91,6800.0,334100000.0,1900000.0,0.0020,665300.0,1.250000e+10,United States,334.1,130.769231,0.0020,1900000.0,5095.743138
4,6,kimkardashian,91,5600.0,329200000.0,3500000.0,0.0088,2900000.0,1.990000e+10,United States,329.2,107.692308,0.0088,3500000.0,9077.982425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,196,iambeckyg,71,2300.0,33200000.0,623800.0,0.0140,464700.0,1.400000e+09,United States,33.2,44.230769,0.0140,623800.0,1612.609662
134,197,nancyajram,81,3800.0,33200000.0,390400.0,0.0064,208000.0,1.500000e+09,France,33.2,73.076923,0.0064,390400.0,1046.411274
135,198,luansantana,79,770.0,33200000.0,193300.0,0.0026,82600.0,1.492000e+08,Brazil,33.2,14.807692,0.0026,193300.0,518.696695
136,199,nickjonas,78,2300.0,33000000.0,719600.0,0.0142,467700.0,1.700000e+09,United States,33.0,44.230769,0.0142,719600.0,1851.949822


## 1. Branch and Bound MILP Formulation

In [69]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from tabulate import tabulate

# Parameters
budget = 90000  # Available budget
min_followers = 100000000  # Minimum number of followers (100M)
k = 3  # Minimum number of different countries
time_available = 6  # Total time available (6 hours)

# Define the weights for the objective function (for simplicity, assuming equal weights here)
w1 = 0.4  # Weight for Influence Score
w2 = 0.3 # Weight for Average Likes
w3 = 0.3  # Weight for Engagement Rate

# Create model
model = gp.Model("Influencer_Selection")

# Decision Variables
x = model.addVars(len(df), vtype=GRB.BINARY, name="x")  # Whether an influencer is selected
t = model.addVars(len(df), vtype=GRB.CONTINUOUS, lb=0, ub=6, name="t")  # Time allocated to each influencer

# Objective Function: Maximize engagement score
model.setObjective(gp.quicksum(
    x[i] * (w1 * df['influence_score'][i] + w2 * df['new_post_avg_like'][i] + w3 * df['60_day_eng_rate'][i])
    for i in range(len(df))
), GRB.MAXIMIZE)

# Constraints

# 1. Budget Constraint: Total cost of selected influencers should not exceed the budget
model.addConstr(gp.quicksum(x[i] * df['Cost_per_Hour'][i] * t[i] for i in range(len(df))) <= budget, "Budget_Constraint")

# 2. Minimum Followers Constraint
model.addConstrs((df['followers'][i] * x[i] >= min_followers * x[i] for i in range(len(df))), "Min_Followers_Constraint")

# 4. Diversity constraint (at least k countries)
countries = list((df['country']))  # Extract unique countries
country_vars = {c: model.addVar(vtype=GRB.BINARY, name=f"country_{c}") for c in set(countries)}
model.addConstrs(
    country_vars[c] <= gp.quicksum(x[i] for i in range(n_influencers) if countries[i] == c)
    for c in set(countries)
)
model.addConstr(gp.quicksum(country_vars[c] for c in set(countries)) >= k, "min_countries")

# 5. Constraint: Select between 2 and 5 influencers
model.addConstr(gp.quicksum(x[i] for i in range(len(df))) >= 2, "Min_Selection")
model.addConstr(gp.quicksum(x[i] for i in range(len(df))) <= 5, "Max_Selection")

# Time Allocation Constraint: Ensure that time allocated is between 1 and 6 hours for selected influencers
model.addConstrs((t[i] >= 1 * x[i] for i in range(len(df))), name="Min_Time_Constraint")
model.addConstrs((t[i] <= 6 * x[i] for i in range(len(df))), name="Max_Time_Constraint")

# Total Time Constraint: The total time spent should sum to exactly 6 hours
model.addConstr(gp.quicksum(t[i] for i in range(len(df))) == time_available, "Total_Time_Constraint")

# Solve the model
model.optimize()

# Get the results
if model.status == GRB.OPTIMAL:
    selected_influencers = []
    total_engagement_score = 0  # Variable to track total engagement score
    total_cost = 0  # Variable to track total cost
    
    for i in range(len(df)):
        if x[i].x > 0.5:  # Influencer selected
            engagement_score = (w1 * df['influence_score'][i] +
                                w2 * df['new_post_avg_like'][i] +
                                w3 * df['60_day_eng_rate'][i])  # Engagement score calculation
            
            cost = df['Cost_per_Hour'][i] * t[i].x  # Cost of selected influencer
            
            total_engagement_score += engagement_score  # Add to total engagement score
            total_cost += cost  # Add to total cost
            
            selected_influencers.append({
                'Rank': df['rank'][i],
                'Influencer': df['channel_info'][i],
                'Time Allocated (hours)': round(t[i].x, 2),  # Round time allocated to 2 decimals
                'Cost': round(cost, 2),  # Round cost to 2 decimals
                'Engagement Score': round(engagement_score, 2),# Round engagement score to 2 decimals
                'Country': df['country'][i]
            })
    
    # Output the results
    selected_influencers_df = pd.DataFrame(selected_influencers)
    print(tabulate(selected_influencers_df, headers='keys', tablefmt='pretty'))
    
    # Print the total engagement score and cost, rounded to 2 decimals
    print(f"\nTotal Engagement Score: {round(total_engagement_score, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}")

else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.4.0 23E214)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 443 rows, 301 columns and 1292 nonzeros
Model fingerprint: 0x2c3e6d47
Model has 1 quadratic constraint
Variable types: 138 continuous, 163 integer (163 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+08]
  QMatrix range    [3e+02, 2e+04]
  Objective range  [3e+01, 2e+06]
  Bounds range     [1e+00, 6e+00]
  RHS range        [2e+00, 6e+00]
  QRHS range       [9e+04, 9e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 385 rows and 247 columns
Presolve time: 0.01s
Presolved: 85 rows, 80 columns, 316 nonzeros
Variable types: 52 continuous, 28 integer (28 binary)
Found heuristic solution: objective 1041176.4068
Found heuristic solution: objective 5130179.6174

Root relaxation: objective 7.260180e+0

#### **Interpretation**:

The objective value (7.26 million) and the bound (7.26 million) are the same, indicating that the optimization process has found an optimal solution with no gap between the best possible outcome and the current solution. 
This means that the model has successfully reached an optimal solution.
The gap of 0.0000% means that the solution is exact and optimal, with no further improvement possible in the context of the given constraints.

#### **Result**

- The model chose 5 influencers: Cristiano Ronaldo, Kylie Jenner, Kendall Jenner, Neymarjr, and Zendaya.
- Time allocation and cost for each influencer vary (e.g., Cristiano Ronaldo was allocated 1.27 hours, while others were given 1 hour each).
- Budget constraint is satisfied (the total cost is exactly 90,000$)
- Minimum followers constraint is satisfied for each selected influencer (>100M followers).
- Diversity constraint is satisfied (at least 3 countries represented).
- Total time allocated sums to exactly 6 hours.

## 2. Branch and Cut MILP Formulation

In [71]:
import gurobipy as gp
from gurobipy import GRB

# Given data (considering the same budget and min followers to compare results)
budget = 90000
min_followers = 100000000
k = 3
w1, w2, w3 = 0.4, 0.3, 0.3

n_influencers = len(df)
influence_scores = df['influence_score'].tolist()
avg_likes = df['new_post_avg_like'].tolist()
engagement_rate = df['60_day_eng_rate'].tolist()
cost_per_hour = df['Cost_per_Hour'].tolist()
countries = df['country'].tolist()

# Model
model = gp.Model("influencer_allocation")

# Variables
x = model.addVars(n_influencers, vtype=GRB.BINARY, name="x")
time_alloc = model.addVars(n_influencers, vtype=GRB.CONTINUOUS, name="time_alloc")

# Objective
model.setObjective(
    gp.quicksum(
        (w1 * influence_scores[i] + w2 * avg_likes[i] + w3 * engagement_rate[i]) * x[i]
        for i in range(n_influencers)
    ), GRB.MAXIMIZE
)

# Constraints
model.addConstr(gp.quicksum(cost_per_hour[i] * time_alloc[i] for i in range(n_influencers)) <= budget, "budget")
model.addConstrs(time_alloc[i] <= 6 * x[i] for i in range(n_influencers))  # Max 6 hours if selected
model.addConstrs(time_alloc[i] >= 1 * x[i] for i in range(n_influencers))  # Min 1 hour if selected
model.addConstr(gp.quicksum(x[i] for i in range(n_influencers)) >= 2, "min_influencers")
model.addConstr(gp.quicksum(x[i] for i in range(n_influencers)) <= 5, "max_influencers")
model.addConstr(gp.quicksum(time_alloc[i] for i in range(n_influencers)) == 6, "total_time") 
model.addConstrs((df['followers'][i] * x[i] >= min_followers * x[i] for i in range(len(df))), "Min_Followers_Constraint")


# Country coverage constraint
country_vars = {c: model.addVar(vtype=GRB.BINARY, name=f"country_{c}") for c in set(countries)}
model.addConstrs(
    country_vars[c] <= gp.quicksum(x[i] for i in range(n_influencers) if countries[i] == c)
    for c in set(countries)
)
model.addConstr(gp.quicksum(country_vars[c] for c in set(countries)) >= k, "min_countries")

# Cutting Plane Logic: Add cuts iteratively
def cutting_plane_callback(model, where):
    if where == GRB.Callback.MIPSOL:
        # Example: Identify fractional solutions and add cuts
        selected_x = model.cbGetSolution([x[i] for i in range(n_influencers)])
        fractional_indices = [i for i, val in enumerate(selected_x) if 0 < val < 1]
        
        if fractional_indices:
            # Add a cut (Example: Simple knapsack cut for fractional solutions)
            model.cbCut(
                gp.quicksum(x[i] for i in fractional_indices) <= len(fractional_indices) - 1
            )

# Optimize with callback
model.optimize(cutting_plane_callback)

# Results
selected_influencers = []
total_engagement_score = 0
total_cost = 0

for i in range(n_influencers):
    if x[i].X > 0.5:  # Check if the influencer is selected
        engagement_score = (w1 * influence_scores[i] +
                            w2 * avg_likes[i] +
                            w3 * engagement_rate[i])
        cost = cost_per_hour[i] * time_alloc[i].X
        selected_influencers.append({
            'Rank': df['rank'][i],
            'Influencer': df['channel_info'][i],
            'Time Allocated (hours)': round(time_alloc[i].X, 2),
            'Cost': round(cost, 2),
            'Engagement Score': round(engagement_score, 2),
            'Country':df['country'][i]
        })
        total_engagement_score += engagement_score
        total_cost += cost

# Convert results to DataFrame
selected_influencers_df = pd.DataFrame(selected_influencers)

# Print results
print("Selected Influencers:")
print(tabulate(selected_influencers_df, headers='keys', tablefmt='pretty'))
print(f"Final Engagement Score: {round(total_engagement_score, 2)}")
print(f"Total Cost: {round(total_cost, 2)}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.4.0 23E214)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 444 rows, 301 columns and 1430 nonzeros
Model fingerprint: 0x72405637
Variable types: 138 continuous, 163 integer (163 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+08]
  Objective range  [3e+01, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 9e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1444702.4081
Presolve removed 385 rows and 247 columns
Presolve time: 0.00s
Presolved: 59 rows, 54 columns, 238 nonzeros
Variable types: 26 continuous, 28 integer (28 binary)

Root relaxation: objective 7.260180e+06, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth I

#### **Interpretation**

The results from the Branch and Bound and Branch and Cut methods are identical in terms of the optimal solution 
(Engagement Score: 7.26 million, Total Cost: $90,000 and Time Allocation for each influencer), there are some key differences in the optimization processes and the outputs that can be highlighted:

#### **Key Differences** :

1. Model Structure and Size:

**Branch and Bound:** The model had 443 rows, 301 columns, and 1292 nonzeros.

**Branch and Cut:** The model had 444 rows, 301 columns, and 1430 nonzeros. This slight difference in the number of rows and nonzeros could indicate some differences in how the models were formulated or processed.

2. Heuristic Solutions:

**Branch and Bound:** Two heuristic solutions were found during the optimization process with objective values of 1,041,176.41 and 5,130,179.62, which is part of the search for an optimal solution.

**Branch and Cut:** One heuristic solution was found with an objective value of 1,444,702.41. This indicates a difference in the heuristic solutions found, though these are just initial solutions and do not affect the final optimal solution.

3. Presolve Time and Matrix Statistics:

**Branch and Bound:** The presolve time was 0.01 seconds, and the matrix had large coefficients. The model was simplified to 85 rows, 80 columns, and 316 nonzeros.

**Branch and Cut:** The presolve time was 0.00 seconds, and the matrix also had large coefficients, but it was simplified to 59 rows, 54 columns, and 238 nonzeros. The difference in the number of rows and columns after presolve suggests that the Branch and Cut method might have been more aggressive in removing redundant constraints or variables during the presolve phase.

## 3. Implementation of Genetic Algorithm

#### a) Considering the best solution without adjusting for the available time

In [97]:
import numpy as np
import pandas as pd
import random
import time 

# Parameters (defined same as above to compare the results)
budget = 90000
min_followers = 100000000
k = 3
time_available = 6  # Total time available (6 hours)
w1, w2, w3 = 0.4, 0.3, 0.3

# Initialize Population
population_size = 100
num_generations = 500
mutation_rate = 0.1

# Create a random initial population where each individual is a binary vector
def create_initial_population():
    population = []
    for _ in range(population_size):
        # Ensure between 2 to 5 influencers are selected at random
        selected_influencers = random.sample(range(len(df)), random.randint(2, 5))
        chromosome = [1 if i in selected_influencers else 0 for i in range(len(df))]
        population.append(np.array(chromosome))
    return population

# Create the fitness function
def fitness(chromosome):
    selected_count = sum(chromosome)  # Count the number of selected influencers
    
    # Ensure the number of selected influencers is between 2 and 5
    if selected_count < 2 or selected_count > 5:
        return -float('inf')  # Penalize infeasible solutions
    
    # Calculate total engagement score, cost, time, and diversity
    engagement_score = 0
    total_cost = 0
    total_time = 0
    country_count = {}
    
    for i in range(len(df)):
        if chromosome[i] == 1:  # Influencer is selected
            # Check for minimum followers
            if df['followers'][i] < min_followers:  
                return -float('inf')  # Penalize influencers with fewer than the minimum followers
            
            engagement_score += (w1 * df['influence_score'][i] +
                                 w2 * df['new_post_avg_like'][i] +
                                 w3 * df['60_day_eng_rate'][i])
            total_cost += df['Cost_per_Hour'][i] * 1  # Assume 1 hour time allocation 
            total_time += 1  # For simplicity, initially assume 1 hour allocated
            
            # Track countries for diversity constraint
            country = df['country'][i]
            country_count[country] = country_count.get(country, 0) + 1
    
    # Apply constraints:
    if total_cost > budget:
        return -float('inf')
    elif total_time > time_available:
        return -float('inf')
    elif len(country_count) < k:
        return -float('inf')  # Penalize infeasible solutions
    
    # Return the engagement score as fitness
    return engagement_score


# Selection Function
def selection(population):
    # Tournament selection
    tournament_size = 5
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(population, tournament_size)
        tournament_fitness = [fitness(individual) for individual in tournament]
        winner = tournament[np.argmax(tournament_fitness)]
        selected.append(winner)
    return selected

# Crossover Function
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(df)-1)
    child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    
    # Ensure child solutions are valid (between 2 and 5 selected influencers)
    while sum(child1) < 2 or sum(child1) > 5:
        crossover_point = random.randint(0, len(df)-1)
        child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    while sum(child2) < 2 or sum(child2) > 5:
        crossover_point = random.randint(0, len(df)-1)
        child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    
    return child1, child2

# Mutation Function
def mutation(chromosome):
    if random.random() < mutation_rate:
        mutation_point = random.randint(0, len(df)-1)
        chromosome[mutation_point] = 1 - chromosome[mutation_point]  # Flip the bit
        
        # Ensure the mutation respects the influencer selection range (2-5)
        selected_count = sum(chromosome)
        if selected_count < 2:
            while selected_count < 2:
                mutation_point = random.randint(0, len(df)-1)
                chromosome[mutation_point] = 1
                selected_count = sum(chromosome)
        elif selected_count > 5:
            while selected_count > 5:
                mutation_point = random.randint(0, len(df)-1)
                chromosome[mutation_point] = 0
                selected_count = sum(chromosome)
                
    return chromosome

# Genetic Algorithm
def genetic_algorithm():
    global population
    population = create_initial_population()  # Initialize population
    
    for generation in range(num_generations):
        # Selection
        selected_population = selection(population)
        
        # Crossover and Mutation
        new_population = []
        for i in range(0, len(selected_population), 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i+1]
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutation(child1))
            new_population.append(mutation(child2))
        
        population = new_population
        
        # Track best solution
        best_solution = max(population, key=fitness)
        best_fitness = fitness(best_solution)
        
        print(f"Generation {generation}, Best Fitness: {best_fitness}")
        
    return best_solution

# Start the timer before the execution of the GA
start_time = time.time()

# Run the Genetic Algorithm
best_solution = genetic_algorithm()

# End the timer after the GA execution completes
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Execution Time: {execution_time:.2f} seconds")

# Extract results for the best solution
selected_influencers = []
total_engagement_score = 0
total_cost = 0
total_time = 0
selected_count = sum(best_solution)

# Ensure the total time is exactly 6 hours and allocate it proportionally
if selected_count < 2 or selected_count > 5:
    print("Selected influencers count is outside the valid range (2 to 5).")
else:
    # Calculate total cost for selected influencers
    total_cost_of_selected = sum([df['Cost_per_Hour'][i] for i in range(len(df)) if best_solution[i] == 1])

    # Apply scaling if total cost exceeds budget
    if total_cost_of_selected > budget:
        scaling_factor = budget / total_cost_of_selected
        total_cost_of_selected = budget  # Ensure we don't exceed the budget
        # Scale the time allocation for each influencer
        time_per_influencer = {}
        for i in range(len(df)):
            if best_solution[i] == 1:
                time_per_influencer[i] = (df['Cost_per_Hour'][i] * scaling_factor) / df['Cost_per_Hour'][i] * time_available
    else:
        # If cost is within budget, allocate time normally
        time_per_influencer = {i: 1 for i in range(len(df)) if best_solution[i] == 1}
    
    # Recalculate the total cost and engagement score using the adjusted time allocation
    for i in range(len(df)):
        if best_solution[i] == 1:  # Influencer selected
            engagement_score = (w1 * df['influence_score'][i] +
                                w2 * df['new_post_avg_like'][i] +
                                w3 * df['60_day_eng_rate'][i])
            cost = df['Cost_per_Hour'][i] * time_per_influencer.get(i, 1)  # Allocate the adjusted time
            total_engagement_score += engagement_score
            total_cost += cost
            total_time += time_per_influencer.get(i, 1)
            
            selected_influencers.append({
                'Rank': df['rank'][i],
                'Influencer': df['channel_info'][i],
                'Time Allocated (hours)': round(time_per_influencer.get(i, 1), 2),
                'Cost': round(cost, 2),
                'Engagement Score': round(engagement_score, 2),
                'Country': df['country'][i]
            })

    # Output the results
    selected_influencers_df = pd.DataFrame(selected_influencers)
    print(tabulate(selected_influencers_df, headers='keys', tablefmt='pretty'))
    print(f"Total Engagement Score: {round(total_engagement_score,2)}")
    print(f"Total Cost: {round(total_cost,2)}")
    print(f"Total Time: {total_time}")

Generation 0, Best Fitness: -inf
Generation 1, Best Fitness: -inf
Generation 2, Best Fitness: -inf
Generation 3, Best Fitness: -inf
Generation 4, Best Fitness: -inf
Generation 5, Best Fitness: -inf
Generation 6, Best Fitness: -inf
Generation 7, Best Fitness: -inf
Generation 8, Best Fitness: -inf
Generation 9, Best Fitness: -inf
Generation 10, Best Fitness: -inf
Generation 11, Best Fitness: -inf
Generation 12, Best Fitness: -inf
Generation 13, Best Fitness: -inf
Generation 14, Best Fitness: -inf
Generation 15, Best Fitness: -inf
Generation 16, Best Fitness: -inf
Generation 17, Best Fitness: -inf
Generation 18, Best Fitness: -inf
Generation 19, Best Fitness: -inf
Generation 20, Best Fitness: -inf
Generation 21, Best Fitness: -inf
Generation 22, Best Fitness: -inf
Generation 23, Best Fitness: -inf
Generation 24, Best Fitness: -inf
Generation 25, Best Fitness: -inf
Generation 26, Best Fitness: -inf
Generation 27, Best Fitness: -inf
Generation 28, Best Fitness: -inf
Generation 29, Best Fitn

#### **Interpretation of the Result:**

The output from the Genetic Algorithm (GA) provides an optimal solution with the following key points:

1. Optimal Solution Found:

The Total Engagement Score of 7.26 million and Total Cost of $78,865.96 were achieved, with a Total Time Allocation of 5 hours. The engagement score and selected influencers are similar to the optimal solution obtained from other methods like Branch and Bound and Branch and Cut, but the cost is slightly lower in this case, suggesting that the Genetic Algorithm was able to allocate time and budget more efficiently.


2. Genetic Algorithm Evolution:
 
The GA was run for 500 generations, which means it evolved and iterated over many possible solutions, refining the selection of influencers each time. This gradual improvement allowed the algorithm to converge on an optimal set of influencers and their time allocations while satisfying all the constraints (budget, time, diversity, and minimum followers).

The tournament selection and crossover processes helped maintain a diverse set of solutions, and mutation ensured that the population did not converge prematurely on suboptimal solutions. Over the generations, the population evolved towards the optimal influencer selection, balancing the engagement score and cost effectively.

#### **Further Improvement:**

Though the optimal solution is similar to the other two methods, we can observe that the total time allocated is just 5 hours, and we still have 1hour to adjust among the influencers given that the total cost did not exceed the given budget. Hence there is still a room for improving the time allocation so that influencers with higher influence score can still be allocated more time, subject to the budget constraint

#### b) Improved Implementation of GA to adjust for the remaining time 

In [105]:
import numpy as np
import pandas as pd
import random

# Parameters (defined same as above to compare results) 
budget = 90000
min_followers = 100000000
k = 3
time_available = 6  # Total time available (6 hours)
w1, w2, w3 = 0.4, 0.3, 0.3

# Genetic Algorithm Parameters
population_size = 100
num_generations = 500
mutation_rate = 0.1

# Repair Function
def repair_chromosome(chromosome):
    selected_indices = [i for i, gene in enumerate(chromosome) if gene == 1]
    unselected_indices = [i for i, gene in enumerate(chromosome) if gene == 0]
    
    # If fewer than 2 influencers are selected
    if len(selected_indices) < 2:
        needed = 2 - len(selected_indices)
        to_select = random.sample(unselected_indices, needed)
        for idx in to_select:
            chromosome[idx] = 1
    
    # If more than 5 influencers are selected
    elif len(selected_indices) > 5:
        excess = len(selected_indices) - 5
        to_deselect = random.sample(selected_indices, excess)
        for idx in to_deselect:
            chromosome[idx] = 0
    
    return chromosome

# Initialize Population
def create_initial_population():
    population = []
    for _ in range(population_size):
        selected_influencers = random.sample(range(len(df)), random.randint(2, 5))
        chromosome = [1 if i in selected_influencers else 0 for i in range(len(df))]
        population.append(repair_chromosome(np.array(chromosome)))
    return population

# Fitness Function
def fitness(chromosome):
    selected_count = sum(chromosome)  # Count the number of selected influencers
    
    # Ensure the number of selected influencers is between 2 and 5
    if selected_count < 2 or selected_count > 5:
        return -float('inf')  # Penalize infeasible solutions
    
    # Calculate total engagement score, cost, time, and diversity
    engagement_score = 0
    total_cost = 0
    total_time = 0
    country_count = {}
    
    for i in range(len(df)):
        if chromosome[i] == 1:  # Influencer is selected
            # Check for minimum followers
            if df['followers'][i] < min_followers:
                return -float('inf')  # Penalize influencers with fewer than the minimum followers
            
            engagement_score += (w1 * df['influence_score'][i] +
                                 w2 * df['new_post_avg_like'][i] +
                                 w3 * df['60_day_eng_rate'][i])
            total_cost += df['Cost_per_Hour'][i]
            total_time += 1  # Assume 1 hour initially
            
            # Track countries for diversity constraint
            country = df['country'][i]
            country_count[country] = country_count.get(country, 0) + 1
    
    # Apply constraints
    if total_cost > budget:
        return -float('inf')
    elif total_time > time_available:
        return -float('inf')
    elif len(country_count) < k:
        return -float('inf')
    
    return engagement_score

# Selection Function
def selection(population):
    tournament_size = 5
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(population, tournament_size)
        tournament_fitness = [fitness(individual) for individual in tournament]
        winner = tournament[np.argmax(tournament_fitness)]
        selected.append(winner)
    return selected

# Crossover Function
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    return repair_chromosome(child1), repair_chromosome(child2)

# Mutation Function
def mutation(chromosome):
    if random.random() < mutation_rate:
        mutation_point = random.randint(0, len(chromosome) - 1)
        chromosome[mutation_point] = 1 - chromosome[mutation_point]  # Flip the bit
    return repair_chromosome(chromosome)

# Genetic Algorithm
def genetic_algorithm():
    global population
    population = create_initial_population()
    
    for generation in range(num_generations):
        selected_population = selection(population)
        new_population = []
        for i in range(0, len(selected_population), 2):
            parent1 = selected_population[i]
            parent2 = selected_population[min(i + 1, len(selected_population) - 1)]  # Handle odd population size
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutation(child1))
            new_population.append(mutation(child2))
        population = [repair_chromosome(individual) for individual in new_population]
        
        # Track the best solution
        best_solution = max(population, key=fitness)
        best_fitness = fitness(best_solution)
        print(f"Generation {generation}, Best Fitness: {best_fitness}")
    
    return best_solution

# Start the timer before the execution of the GA
start_time = time.time()

# Run the Genetic Algorithm
best_solution = genetic_algorithm()

# End the timer after the GA execution completes
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Execution Time: {execution_time:.2f} seconds")

# Step 1: Assign initial time (1 hour each for selected influencers)
time_allocation = {}
total_cost = 0
total_time = 0
selected_influencers = []
initial_time_allocation = {}

# Assign 1 hour to each selected influencer and calculate total cost and time
for i in range(len(df)):
    if best_solution[i] == 1:
        time_allocation[i] = 1
        initial_time_allocation[i] = 1  # Initially 1 hour per influencer
        total_time += 1  # Sum up the total time (which will be 5 initially)
        influencer_cost = df['Cost_per_Hour'][i] * time_allocation[i]
        total_cost += influencer_cost

# Step 2: Check if the total cost exceeds the budget
# If the total cost is within budget, try to increase the time for influencers with high engagement scores
if total_cost < budget:
    # Sort influencers by their influence score (descending)
    df_sorted = df[best_solution == 1].sort_values(by=['influence_score'], ascending=False)

    for i in range(len(df_sorted)):
        influencer_index = df_sorted.index[i]  # Get the original index
        max_possible_time = (budget - total_cost) / df['Cost_per_Hour'][influencer_index]
        
        # Calculate the maximum time we can allocate without exceeding the budget
        if total_time < time_available:
            additional_time = min(max_possible_time, time_available - total_time)
            time_allocation[influencer_index] += additional_time
            total_time += additional_time
            total_cost += df['Cost_per_Hour'][influencer_index] * additional_time

            # If the total cost exceeds the budget, stop adjusting the time
            if total_cost > budget:
                break

# Step 3: Balance time allocation if total time exceeds available time
if total_time > time_available:
    # Scale down the time allocation proportionally
    scaling_factor = time_available / total_time
    for i in range(len(df)):
        if best_solution[i] == 1:
            time_allocation[i] *= scaling_factor

# Step 4: Calculate the final total cost and engagement scores
total_cost = 0
total_engagement_score = 0
selected_influencers = []

for i in range(len(df)):
    if best_solution[i] == 1:
        # Recalculate the cost for each influencer based on the adjusted time allocation
        influencer_cost = df['Cost_per_Hour'][i] * time_allocation[i]
        total_cost += influencer_cost
        
        # Calculate the engagement score
        engagement_score = (w1 * df['influence_score'][i] +
                            w2 * df['new_post_avg_like'][i] +
                            w3 * df['60_day_eng_rate'][i])
        total_engagement_score += engagement_score
        
        # Store the selected influencer details
        selected_influencers.append({
            'Rank': df['rank'][i],
            'Influencer': df['channel_info'][i],
            'Time Allocated (hours)': round(time_allocation[i], 2),
            'Cost': round(influencer_cost, 2),
            'Engagement Score': round(engagement_score, 2),
            'Country': df['country'][i],
        })

# Output the results
selected_influencers_df = pd.DataFrame(selected_influencers)
print(tabulate(selected_influencers_df, headers='keys', tablefmt='pretty'))
print(f"Total Engagement Score: {round(total_engagement_score,2)}")
print(f"Total Cost: {round(total_cost,2)}")
print(f"Total Time: {sum(time_allocation.values())}")

Generation 0, Best Fitness: -inf
Generation 1, Best Fitness: -inf
Generation 2, Best Fitness: -inf
Generation 3, Best Fitness: -inf
Generation 4, Best Fitness: -inf
Generation 5, Best Fitness: 2006594.81425
Generation 6, Best Fitness: 2006594.81425
Generation 7, Best Fitness: 2006594.81425
Generation 8, Best Fitness: 2006594.81425
Generation 9, Best Fitness: 3956631.61842
Generation 10, Best Fitness: 3956631.61842
Generation 11, Best Fitness: 3956631.61842
Generation 12, Best Fitness: 4766668.4220199995
Generation 13, Best Fitness: 4946668.82133
Generation 14, Best Fitness: 4946668.82133
Generation 15, Best Fitness: 4946668.82133
Generation 16, Best Fitness: 4946668.82133
Generation 17, Best Fitness: 4946668.82133
Generation 18, Best Fitness: 4946668.82133
Generation 19, Best Fitness: 4946668.82133
Generation 20, Best Fitness: 4946668.82133
Generation 21, Best Fitness: 4946668.82133
Generation 22, Best Fitness: 5190178.42175
Generation 23, Best Fitness: 5190178.42175
Generation 24, Bes

### **Interpretation:**

By allocating more time to the influencer with higher influence_score, we achieved the same result as the previous methods. However, we can still see that the total available time (6 hours) was not utilized completely as the primary goal was to satisfy the budget constraint and to increase the time based on the influencer's score. 

#### **Improvement:**

This implementation can be modified into a different application to get the approximate estimation of additional cost incurred to accomodate the influencers for all the available 6 hours. 

### c) Implementation of GA to improve the time allocation and estimating the additional cost required 

In [125]:
import numpy as np
import pandas as pd
import random

# Parameters (defined same as above to compare results)
budget = 90000
min_followers = 100000000
k = 3
time_available = 6  # Total time available (6 hours)
w1, w2, w3 = 0.4, 0.3, 0.3

# Initialize Population
population_size = 100
num_generations = 500
mutation_rate = 0.1

# Create a random initial population where each individual is a binary vector
def create_initial_population():
    population = []
    for _ in range(population_size):
        # Ensure between 2 to 5 influencers are selected at random
        selected_influencers = random.sample(range(len(df)), random.randint(2, 5))
        chromosome = [1 if i in selected_influencers else 0 for i in range(len(df))]
        population.append(np.array(chromosome))
    return population

def fitness(chromosome):
    selected_count = sum(chromosome)  # Count the number of selected influencers
    
    # Ensure the number of selected influencers is between 2 and 5
    if selected_count < 2 or selected_count > 5:
        return -float('inf')  # Penalize infeasible solutions
    
    # Calculate total engagement score, cost, time, and diversity
    engagement_score = 0
    total_cost = 0
    total_time = 0
    country_count = {}
    
    for i in range(len(df)):
        if chromosome[i] == 1:  # Influencer is selected
            # Check for minimum followers
            if df['followers'][i] < min_followers:  
                return -float('inf')  # Penalize influencers with fewer than the minimum followers
            
            engagement_score += (w1 * df['influence_score'][i] +
                                 w2 * df['new_post_avg_like'][i] +
                                 w3 * df['60_day_eng_rate'][i])
            total_cost += df['Cost_per_Hour'][i] * 1  # Assume 1 hour time allocation for now
            total_time += 1  # For simplicity, initially assume 1 hour allocated
            
            # Track countries for diversity constraint
            country = df['country'][i]
            country_count[country] = country_count.get(country, 0) + 1
    
    # Apply constraints:
    if total_cost > budget:
        return -float('inf')
    elif total_time > time_available:
        return -float('inf')
    elif len(country_count) < k:
        return -float('inf')  # Penalize infeasible solutions
    
    # Return the engagement score as fitness
    return engagement_score


# Selection Function
def selection(population):
    # Tournament selection
    tournament_size = 5
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(population, tournament_size)
        tournament_fitness = [fitness(individual) for individual in tournament]
        winner = tournament[np.argmax(tournament_fitness)]
        selected.append(winner)
    return selected

# Crossover Function
def crossover(parent1, parent2):
    crossover_point = random.randint(0, len(df)-1)
    child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    
    # Ensure child solutions are valid (between 2 and 5 selected influencers)
    while sum(child1) < 2 or sum(child1) > 5:
        crossover_point = random.randint(0, len(df)-1)
        child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    while sum(child2) < 2 or sum(child2) > 5:
        crossover_point = random.randint(0, len(df)-1)
        child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
    
    return child1, child2

# Mutation Function
def mutation(chromosome):
    if random.random() < mutation_rate:
        mutation_point = random.randint(0, len(df)-1)
        chromosome[mutation_point] = 1 - chromosome[mutation_point]  # Flip the bit
        
        # Ensure the mutation respects the influencer selection range (2-5)
        selected_count = sum(chromosome)
        if selected_count < 2:
            while selected_count < 2:
                mutation_point = random.randint(0, len(df)-1)
                chromosome[mutation_point] = 1
                selected_count = sum(chromosome)
        elif selected_count > 5:
            while selected_count > 5:
                mutation_point = random.randint(0, len(df)-1)
                chromosome[mutation_point] = 0
                selected_count = sum(chromosome)
                
    return chromosome

# Genetic Algorithm
def genetic_algorithm():
    global population
    population = create_initial_population()  # Initialize population
    
    for generation in range(num_generations):
        # Selection
        selected_population = selection(population)
        
        # Crossover and Mutation
        new_population = []
        for i in range(0, len(selected_population), 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i+1]
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutation(child1))
            new_population.append(mutation(child2))
        
        population = new_population
        
        # Track best solution
        best_solution = max(population, key=fitness)
        best_fitness = fitness(best_solution)
        
        print(f"Generation {generation}, Best Fitness: {best_fitness}")
        
    return best_solution

# Start the timer before the execution of the GA
start_time = time.time()

# Run the Genetic Algorithm
best_solution = genetic_algorithm()

# End the timer after the GA execution completes
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Execution Time: {execution_time:.2f} seconds")

# Extract results for the best solution
selected_influencers = []
total_engagement_score = 0
total_cost = 0
total_time = 0
selected_count = sum(best_solution)

# Ensure the total time is exactly 6 hours and allocate it proportionally based on cost
if selected_count < 2 or selected_count > 5:
    print("Selected influencers count is outside the valid range (2 to 5).")
else:
    # Calculate total cost for selected influencers
    total_cost_of_selected = sum([df['Cost_per_Hour'][i] for i in range(len(df)) if best_solution[i] == 1])
    
    # Allocate time proportionally to each influencer based on their cost
    time_per_influencer = {}
    for i in range(len(df)):
        if best_solution[i] == 1:
            time_per_influencer[i] = (df['Cost_per_Hour'][i] / total_cost_of_selected) * time_available
    
    for i in range(len(df)):
        if best_solution[i] == 1:  # Influencer selected
            engagement_score = (w1 * df['influence_score'][i] +
                                w2 * df['new_post_avg_like'][i] +
                                w3 * df['60_day_eng_rate'][i])
            cost = df['Cost_per_Hour'][i] * time_per_influencer[i]  # Allocate the calculated time
            total_engagement_score += engagement_score
            total_cost += cost
            total_time += time_per_influencer[i]
            
            selected_influencers.append({
                'Rank': df['rank'][i],
                'Influencer': df['channel_info'][i],
                'Time Allocated (hours)': round(time_per_influencer[i], 2),
                'Cost': round(cost, 2),
                'Engagement Score': round(engagement_score, 2),
                'Country': df['country'][i]
            })

    # Output the results
    selected_influencers_df = pd.DataFrame(selected_influencers)
    print(tabulate(selected_influencers_df, headers='keys', tablefmt='pretty'))
    print(f"Total Engagement Score: {round(total_engagement_score,2)}")
    print(f"Total Cost: {round(total_cost,2)}")
    print(f"Total Time: {round(total_time,2)}")
    if total_cost>budget:
        print(f"Additional budget required for this allocation : {round((total_cost-budget),2)}")

Generation 0, Best Fitness: -inf
Generation 1, Best Fitness: -inf
Generation 2, Best Fitness: -inf
Generation 3, Best Fitness: -inf
Generation 4, Best Fitness: -inf
Generation 5, Best Fitness: -inf
Generation 6, Best Fitness: -inf
Generation 7, Best Fitness: -inf
Generation 8, Best Fitness: -inf
Generation 9, Best Fitness: -inf
Generation 10, Best Fitness: -inf
Generation 11, Best Fitness: -inf
Generation 12, Best Fitness: -inf
Generation 13, Best Fitness: -inf
Generation 14, Best Fitness: -inf
Generation 15, Best Fitness: -inf
Generation 16, Best Fitness: -inf
Generation 17, Best Fitness: -inf
Generation 18, Best Fitness: -inf
Generation 19, Best Fitness: -inf
Generation 20, Best Fitness: -inf
Generation 21, Best Fitness: -inf
Generation 22, Best Fitness: -inf
Generation 23, Best Fitness: -inf
Generation 24, Best Fitness: -inf
Generation 25, Best Fitness: -inf
Generation 26, Best Fitness: -inf
Generation 27, Best Fitness: -inf
Generation 28, Best Fitness: -inf
Generation 29, Best Fitn

#### **Interpretation:**

The total Engagement Score remains the same at 7.26million which indicates that the selected influencers continue to generate the same level of engagement. This is crucial because the goal is to maximize the engagement score given a fixed budget and time.

The Total Time is now 6.0 hours, which perfectly matches the available time to allocate to influencers. This shows that the time allocation has been optimized to fully utilize the available 6 hours.

The Additional Budget Required is 16,190.84, meaning that the original budget of 90,000 is not sufficient to cover the increased costs of allocating time to these influencers. The adjusted allocation requires an extra 16,190.84 to maintain the engagement levels and utilize the available time effectively.

The result shows that while the time is fully utilized, it requires a slightly higher financial commitment to achieve this optimal allocation. However, this could give an estimation of extra costs, if the company decides to utilize all the available time. 

### **Conclusion:**

While the Genetic Algorithm offers a robust approach for solving complex optimization problems, there were several drawbacks in its implementation for the influencer selection and time allocation task. These drawbacks included longer execution times, the need for additional logic to handle time allocation, inefficient use of available time, and variability in results across executions.

1. The algorithm took longer to produce the output (more than 10 seconds in every case). This can be attributed to the nature of genetic algorithms, which involve iterative processes and population evaluations, making them computationally expensive, especially for larger datasets.

2. After selecting the best solution (the optimal set of influencers), additional logic was needed to handle the time allocation efficiently. This required manual adjustments or more complex logic to ensure that time was distributed properly across influencers, which added to the complexity of the implementation.

3. Despite selecting the best influencers, the algorithm couldn't completely allocate the available 6 hours. In your case, only 5.5 hours were utilized. This indicates that the genetic algorithm did not optimize the time allocation as effectively as possible, potentially leaving unused time, which could have been spent on additional influencers.

4. If the time allocated is adjusted to utilize the available time, we can observe an increase in the budget. However, this can be used to estimate the additional costs incurred if the influencers have to utilized for the entire time.

5. The results were different for each execution, which is a common issue with genetic algorithms due to their random nature. The algorithm’s performance can vary because it relies on random initialization and stochastic processes (such as crossover and mutation), leading to different results with each run. The solution obtained may not be the optimal soluton for every execution. 